In [2]:
import os

import ollama
import chromadb
from langchain_community.document_loaders import PyPDFLoader

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

In [3]:
from utils.markdown_to_string import markdown_file_to_string

In [4]:
# ollama need to be created befoer runing 
embeddings = OllamaEmbeddings(model="hsbi")

vector_directory = 'testing3'
os.makedirs(vector_directory, exist_ok=True)

client = chromadb.PersistentClient(path=vector_directory)


In [5]:
# if the collection is not created
collection = client.create_collection(name="docs")

In [6]:
# collection = client.get_collection(name="docs")

In [7]:
import nest_asyncio

nest_asyncio.apply()

from llama_parse import LlamaParse

In [11]:

parser = LlamaParse(
    api_key="llx-VQsncrO19nFgA9cqAIxguyYViqbuc3XhxfwdRaUXm5Y4Mawf",  
    result_type="markdown",  # "markdown" and "text" are available
    num_workers=4,  # if multiple files passed, split in `num_workers` API calls
    verbose=True,
    language="en",  # Optionally you can define a language, default=en
)

path_monthly_return = "./src/2024070301674.pdf"
path_financial_report = "./src/2023042801118.pdf"
path_sdi = "./src/test.csv"


documents = parser.load_data([
    path_monthly_return,
    path_financial_report,
])
text = [document.text for i, document in enumerate(documents)]
text.append( markdown_file_to_string(path_sdi) )

Parsing files: 100%|██████████| 2/2 [00:48<00:00, 24.30s/it]


In [12]:
for i, d in enumerate(text):
  response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[str(i)],
    embeddings=[embedding],
    documents=[d]
  )

In [15]:
prompt = '''
What is the Number of issued shares (excluding treasury shares)of Logan Group Company Limited, balance at close of the month?(03380), which shouldn't is not with the authorized/registered shares
'''
prompt_4_embedding_search = ''''# III. Details of Movements in Issued Shares and/or Treasury Shares
'''


# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt_4_embedding_search,
  model="mxbai-embed-large"
)
# n_results is the number of page that the vector db take out
results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=10
)
data = results['documents'][0][0]


output = ollama.generate(
  model="llama3",
  prompt=f"Using this data: {data}. Respond to this prompt: {prompt}"
)

print(output['response'])
# retriever = embeddings_store.as_retriever()
print()
#
# splits = ["testing", "testing 2 ", "testing3"]
#
# # Embed the text splits into the vector database
# embeddings_store.from_texts(splits)
#
# # Create a retriever object
# retriever = embeddings_store.as_retriever()
# # Use the retriever to retrieve similar vectors
# similar_vectors = retriever.retrieve("query_text")
# print(similar_vectors)

According to the data, the Number of issued shares (excluding treasury shares) of Logan Group Company Limited (03380) as of the end of the month is **5,685,407,450**.


In [16]:
prompt = '''
How many Number of employee of Logan Group Company Limited?
'''
prompt_4_embedding_search = '''' exployee
'''


# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt_4_embedding_search,
  model="mxbai-embed-large"
)

results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=10
)
data = results['documents'][0][0]

output = ollama.generate(
  model="llama3",
  prompt=f"Using this data: {data}. Respond to this prompt: {prompt}"
)

print(output['response'])


Based on the provided data, there is no information about the number of employees of Logan Group Company Limited. The report only provides biographical information and roles of the executive directors and senior management members, but it does not provide any information about the total number of employees or staff size of the company.
